<a href="https://colab.research.google.com/github/msmsm104/Dacon/blob/TIL/sampling/OverSampling_(RandomOverSampler)_0616.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
import random
import pandas as pd
import numpy as np
import os
import librosa

from tqdm.auto import tqdm

from imblearn.over_sampling import *

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import warnings
warnings.filterwarnings(action='ignore') 

## Load Data

In [62]:
def preprocessing(file_train):
    train_df = pd.read_csv(file_train)
    
    train_x = train_df.drop(columns=['id', 'covid19'])
    train_y = train_df['covid19']
    
    def onehot_encoding(ohe, x):
        # 학습데이터로 부터 fit된 one-hot encoder (ohe)를 받아 transform 시켜주는 함수
        encoded = ohe.transform(x['gender'].values.reshape(-1,1))
        encoded_df = pd.DataFrame(encoded, columns=ohe.categories_[0])
        x = pd.concat([x.drop(columns=['gender']), encoded_df], axis=1)
        return x

    # 'gender' column의 경우 추가 전처리가 필요 -> OneHotEncoder 적용
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(train_x['gender'].values.reshape(-1,1))
    train_x = onehot_encoding(ohe, train_x)
    
    
    model = MLPClassifier(random_state=41)
    model.fit(train_x, train_y)
    
    pred = model.predict(train_x)
    print(f"accuracy score: {accuracy_score(train_y, pred)}")
    print(confusion_matrix(train_y, pred))
    print(f1_score(train_y, pred))
    

In [63]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/이어드림스쿨/TIL(Today I Learned)/DaconProject/EDA/data/train_mfcc_data(32).csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/이어드림스쿨/TIL(Today I Learned)/DaconProject/EDA/data/test_mfcc_data(32).csv')

In [64]:
train_x = train_df.drop(columns=['id', 'covid19'])
train_y = train_df['covid19']
test_x = test_df.drop(columns=['id'])

In [65]:
def onehot_encoding(ohe, x):
        # 학습데이터로 부터 fit된 one-hot encoder (ohe)를 받아 transform 시켜주는 함수
        encoded = ohe.transform(x['gender'].values.reshape(-1,1))
        encoded_df = pd.DataFrame(encoded, columns=ohe.categories_[0])
        x = pd.concat([x.drop(columns=['gender']), encoded_df], axis=1)
        return x

In [66]:
# 'gender' column의 경우 추가 전처리가 필요 -> OneHotEncoder 적용
ohe = OneHotEncoder(sparse=False)
ohe.fit(train_x['gender'].values.reshape(-1,1))
train_x = onehot_encoding(ohe, train_x)

test_x = onehot_encoding(ohe, test_x)

In [67]:
X_samp, y_samp = RandomOverSampler(random_state=0).fit_resample(train_x, train_y)

In [76]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = \
train_test_split(X_samp, y_samp, test_size=0.2, random_state=100)

In [77]:
model = MLPClassifier(random_state=41)
model.fit(X_train, y_train)
    
pred = model.predict(X_val)
print(f"accuracy score: {accuracy_score(y_val, pred)}")
print(confusion_matrix(y_val, pred))
print(f1_score(y_val, pred))

accuracy score: 0.8871428571428571
[[598 125]
 [ 33 644]]
0.8907330567081605


In [74]:
pred = model.predict(test_x)

submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/이어드림스쿨/TIL(Today I Learned)/DaconProject/EDA/data/sample_submission.csv')
submission['covid19'] = pred
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/이어드림스쿨/TIL(Today I Learned)/DaconProject/EDA/data/submit_0616.csv', index=False)

## => test 결과가 아주 좋게 나올것으로 예측 했지만 오히려 성능이 떨어짐.

### MFCC 32

In [ ]:
preprocessing('./Interim/train_mfcc_data.csv')



accuracy score: 0.9038107752956636
[[3348  151]
 [ 215   91]]
0.3321167883211679


In [34]:
model = MLPClassifier(random_state=41)
model.fit(X_samp, y_samp)
    
pred = model.predict(X_samp)
print(f"accuracy score: {accuracy_score(y_samp, pred)}")
print(confusion_matrix(y_samp, pred))
print(f1_score(y_samp, pred))

accuracy score: 0.9329808516719063
[[3152  347]
 [ 122 3377]]
0.9350685310812681


### without gender column

In [ ]:
preprocessing_ecpt_gender('./Interim/train_mfcc_data.csv')

accuracy score: 0.9219448094612352
[[3496    3]
 [ 294   12]]
0.07476635514018691


### without age column

In [ ]:
preprocessing_ecpt_age('./Interim/train_mfcc_data.csv')



accuracy score: 0.9082785808147175
[[3383  116]
 [ 233   73]]
0.29494949494949496


### without age, gender

In [ ]:
preprocessing_ecpt_both('./Interim/train_mfcc_data.csv')

accuracy score: 0.9235216819973718
[[3438   61]
 [ 230   76]]
0.3431151241534989


 => 학습데이터에 대해 비교하였을때, (나이, 성별)을 제외하고 모델을 학습, 그에따른 학습 성능을 평가했을때, 둘 다 포함한 경우 보다 모두 f1-score가 낮게 관측된다.